In [1]:
import sys
import os
import pandas as pd
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

# Adicionar o diretório 'database' ao path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../Banco_de_dados')))
from db import *

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../Funcs_principais')))
from arquivo import save_clusters


#### Recupera os likes e views do BD, calcula engajamento e clusteriza

In [ ]:


data = get_all_data("../Banco_de_dados/videos_data.db","links","likes","views")

# Criar DataFrame
df = pd.DataFrame(data, columns=["ID", "likes", "views"])

# Calcular Engajamento (likes/views)
df['engajamento'] = df['likes'] / df['views']

# Converter para formato numérico para clustering
X = df[['engajamento']].values

# Aplicar K-Means com x clusters
kmeans = KMeans(n_clusters=5, random_state=43, n_init=10)
df['cluster'] = kmeans.fit_predict(X)

cluster_means = df.groupby('cluster')['engajamento'].mean().sort_values()
# Criar um mapeamento antigo → novo
cluster_mapping = {old_label: new_label for new_label, old_label in enumerate(cluster_means.index)}
df['cluster_ordenado'] = df['cluster'].map(cluster_mapping)


# Visualizar os resultados
plt.scatter(df['engajamento'], df['cluster_ordenado'], cmap='viridis', edgecolors='k')
plt.xlabel('Engajamento')
plt.ylabel('Cluster')
plt.title('Clusterização de Vídeos por Engajamento')
plt.show()


In [ ]:
push_many_data("../Banco_de_dados/videos_data.db", "video_clusters", "cluster_engage",df[["ID", "cluster_ordenado"]].values.tolist())
save_clusters(df, "engage_clusters")

#### Recupera as transcrições do BD e clusteriza

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

data = get_all_data("../Banco_de_dados/videos_data.db","transcripts","transcription_normalizado")

# Criar DataFrame
df = pd.DataFrame(data, columns=["ID", "transcription"])

df_filtered = df[df['transcription'] != '']
df_sem_transcription = df[df['transcription'] == '']

df_sem_transcription['cluster'] = -1

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df_filtered['transcription'])

kmeans = KMeans(n_clusters=5, random_state=43)  # Ajuste o número de clusters conforme necessário
df_filtered['cluster'] = kmeans.fit_predict(X)

df_final = pd.concat([df_sem_transcription, df_filtered], ignore_index=True)

# Salvando cada cluster em um arquivo CSV
save_clusters(df_final, "transcriptions_clusters")
push_many_data("../Banco_de_dados/videos_data.db", "video_clusters", "cluster_transcript",df_final[["ID", "cluster"]].values.tolist())


#### Recupera as captions, agrupa por video e clusteriza

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import string

data = get_all_data("../Banco_de_dados/videos_data.db","captions","captions_normalizado","VideoID")

# Criar DataFrame
df = pd.DataFrame(data, columns=["ID", "captions", "VideoID"])

# Agrupar os frames por vídeo
grouped_frames = df.groupby("VideoID")["captions"].apply(list).reset_index()

# Join the captions for each video into a single string
grouped_frames['captions'] = grouped_frames['captions'].apply(lambda x: ' '.join(x))

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(grouped_frames['captions'])

kmeans = KMeans(n_clusters=5, random_state=43)  # Ajuste o número de clusters conforme necessário
grouped_frames['cluster'] = kmeans.fit_predict(X)

# Criar um mapeamento de números para letras (A, B, C, ...)
cluster_labels = {new_label: letter for new_label, letter in enumerate(string.ascii_uppercase)}

# Aplicar o mapeamento
grouped_frames['cluster_letra'] = grouped_frames['cluster'].map(cluster_labels)

print(grouped_frames['cluster_letra'])  # Ver o mapeamento feito




In [ ]:
# Salvando cada cluster em um arquivo CSV
save_clusters(grouped_frames, "captions_clusters")

push_many_data("../Banco_de_dados/videos_data.db", "video_clusters", "cluster_caption",grouped_frames[["VideoID", "cluster_letra"]].values.tolist())

#### Clusterização global

In [ ]:
from kmodes.kmodes import KModes
import matplotlib.pyplot as plt
import pandas as pd

data = get_all_data("../Banco_de_dados/videos_data.db","video_clusters","cluster_engage","cluster_transcript","cluster_caption")

df = pd.DataFrame(data, columns=["ID", "cluster_engage", "cluster_transcript", "cluster_caption"])

# Definir número de clusters
n_clusters = 9
km = KModes(n_clusters=n_clusters, init='Huang', random_state=42)

# Aplicar o modelo aos clusters existentes
df["cluster"] = km.fit_predict(df[["cluster_engage", "cluster_transcript", "cluster_caption"]])




In [ ]:
save_clusters(df, "global_cluster")
push_many_data("../Banco_de_dados/videos_data.db", "video_clusters", "cluster_global",df[["ID", "cluster"]].values.tolist())

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(df['cluster_engage'], df['cluster'], c=df['cluster_engage'], cmap='viridis', edgecolors='k')
plt.xlabel('Cluster de Engajamento')
plt.ylabel('Cluster Final dos Vídeos')
plt.title('Distribuição dos Vídeos nos Clusters')
plt.colorbar(label='Cluster de Engajamento')
plt.show()